In [2]:
%pip install langchain langchain_community openai python-dotenv duckduckgo-search  google-api-python-client 

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os
load_dotenv() 
os.environ['GOOLE_API_KEY'][:5], os.environ['GOOGLE_CSE_ID'][:5]

('AIzaS', 'b3779')

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchResults
from googleapiclient.discovery import build

# OpenAI LLM 설정
llm = OpenAI(temperature=0)



def google_search(query, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(
        q=query,
        cx=cse_id,
        lr='lang_ko',
        hl='ko',
        gl='kr'
    ).execute()
    items = res.get('items', [])
    results_text = []
    for item in items:
        title = item.get('title')
        snippet = item.get('snippet')
        link = item.get('link')
        results_text.append(f"제목: {title}\n요약: {snippet}\n링크: {link}\n")
    return "\n\n".join(results_text)

# Tool 등록 예
from langchain.agents import Tool

tools = [
    Tool(
        name="GoogleSearch",
        func=lambda q: google_search(q, os.environ['GOOLE_API_KEY'], os.environ['GOOGLE_CSE_ID']),
        description="입력된 한글 검색어로 한국 웹사이트를 검색합니다. 쿼리는 변형하지 말고 그대로 사용하세요."
    )
]

# RAG 에이전트 초기화
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# 사용자의 질문에 대해 응답 처리
user_input = "2025년 대선 후보"
try:
    response = agent.run(user_input)
except Exception as e:
    print(f'관련 자료를 찾을수 없습니다. : \n\n{e}')
else:
    print(response)


In [6]:
def google_search(query, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(
        q=query,
        cx=cse_id,
        lr='lang_ko',
        hl='ko',
        gl='kr'
    ).execute()
    items = res.get('items', [])
    results_text = []
    for item in items:
        title = item.get('title')
        snippet = item.get('snippet')
        link = item.get('link')
        results_text.append(f"제목: {title}\n요약: {snippet}\n링크: {link}\n")
    # return "\n\n".join(results_text)
    return results_text

q = '2025년 대선'
for result in google_search(q, os.environ['GOOLE_API_KEY'], os.environ['GOOGLE_CSE_ID']):
    print(result)


제목: 제21대 대통령 선거 - 나무위키
요약: 2025년 5월 19일, 제21대 대통령 선거 책자형 1차 선거공보가 각 유권자들에게 발송되었다. 내용물은 각 후보별 홍보 안내문이 1부씩 첨부되었는데 양대정당 출마자들인 ...
링크: https://namu.wiki/w/%EC%A0%9C21%EB%8C%80%20%EB%8C%80%ED%86%B5%EB%A0%B9%20%EC%84%A0%EA%B1%B0

제목: 대한민국 제21대 대통령 선거 - 위키백과, 우리 모두의 백과사전
요약: 선거일은 2025년 4월 8일에 열린 국무회의에서 2025년 6월 3일로 확정되었다.
링크: https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%EC%A0%9C21%EB%8C%80_%EB%8C%80%ED%86%B5%EB%A0%B9_%EC%84%A0%EA%B1%B0

제목: 2025 대선: 제21대 대통령 선거, 확정된 대선 후보들은 누구? - BBC ...
요약: 2025. 5. 11. ... ... 2025년 5월 11일. 2025년 5월 21일 에 업데이트 됨. 오는 6월 3일 치러지는 제21대 대통령 선거를 앞두고 대선 후보자 명단이 확정됐다. 중앙선거관리 ...
링크: https://www.bbc.com/korean/articles/ckg4npzn88eo

제목: 2025 대선 총정리 - 21대 대통령 후보 이력과 주요 뉴스 업데이트
요약: 2010~2018년: 경기도 성남시장. 2018~2021년: 경기도 도지사. 2022년: 제20대 대선 더불어민주당 후보. 2022년~현재: 국회의원(인천 계양 을). 2022~2025년: 더불어민주당 ...
링크: https://newneek.co/issue/election2025

제목: 21대 대선: '6월 3일' 제21대 대통령 선거 확정…대권 주자는 누구 ...
요약: 2025. 4. 8. ... '6월 3일' 제21대 대통령 선거일 확정…대권 주자는? · 기자, 이